In [1]:

import os, sys
sys.path.append(os.getcwd())

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.utils import get_summary_stats, pick_category_column

In [5]:
arr = np.arange(1, 1_000_001)

vec_squared = arr ** 2

import time

t0 = time.time()
_ = arr ** 2
t1 = time.time()
vec_time = t1 - t0

t0 = time.time()
loop_out = np.empty_like(arr)
for i, v in enumerate(arr):
    loop_out[i] = v * v
t1 = time.time()
loop_time = t1 - t0

vec_time, loop_time


(0.0020639896392822266, 0.15109658241271973)

In [6]:
df = pd.read_csv("data/starter_data.csv")

display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  10 non-null     object
 1   value     10 non-null     int64 
 2   date      10 non-null     object
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


None

,category,value,date
0,A,10,2025-08-01
1,B,15,2025-08-02
2,A,12,2025-08-03
3,B,18,2025-08-04
4,C,25,2025-08-05


In [12]:
df_num = df.copy()
for c in df_num.columns:
    if df_num[c].dtype == "object":
        try:
            df_num[c] = pd.to_numeric(df_num[c])
        except Exception:
            pass

num_only = df_num.select_dtypes(include="number")
summary_df = get_summary_stats(num_only) if not num_only.empty else pd.DataFrame()
display(summary_df if not summary_df.empty else "No numeric columns for describe().")

cat_col = pick_category_column(df_num)
numeric_cols = df_num.select_dtypes(include="number").columns

if len(numeric_cols) == 0:
    grouped = df_num.groupby(cat_col).size().to_frame("count")
else:
    grouped = df_num.groupby(cat_col)[numeric_cols].agg(['count', 'mean', 'std', 'min', 'max'])

display(grouped.head())

,count,mean,std,min,25%,50%,75%,max
value,10.0,17.6,7.381659,10.0,12.25,14.5,23.25,30.0


value                             
         count       mean       std min max
category                                   
A            4  11.500000  1.290994  10  13
B            3  15.666667  2.081666  14  18
C            3  27.666667  2.516611  25  30

In [13]:
os.makedirs("data/processed", exist_ok=True)
summary_path = "data/processed/summary.csv"
summary_df.to_csv(summary_path, index=True)

if cat_col in df.columns:
    counts = df[cat_col].value_counts().sort_values(ascending=False)
    plt.figure()
    counts.plot(kind='bar', title=f"Counts by {cat_col}")
    plt.tight_layout()
    plt.savefig("data/processed/basic_plot.png")
    plt.close()
else:
    num_cols = df.select_dtypes(include='number').columns
    if len(num_cols) > 0:
        plt.figure()
        df[num_cols[0]].plot(kind='hist', bins=30, title=f"Histogram of {num_cols[0]}")
        plt.tight_layout()
        plt.savefig("data/processed/basic_plot.png")
        plt.close()

summary_path

'data/processed/summary.csv'

In [14]:
_ = get_summary_stats(df).head()
_

,count,mean,std,min,25%,50%,75%,max
value,10.0,17.6,7.381659,10.0,12.25,14.5,23.25,30.0
